In [1]:
%reload_ext autoreload
%autoreload 2
from midi_api import initialize_pygame
import time, pygame.midi
from musenet_api import decode_musenet, musenet_generate, encode_musenet

initialize_pygame()

pygame 2.1.2 (SDL 2.0.18, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)


## Interactive

In [2]:
# Get input
from midi_api import input_main 
initialize_pygame()
midi_notes = input_main(1, max_time=7500)
print("Recorded", len(midi_notes))

Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
using input_id :1:
8554
Time is up
Recorded 29


In [3]:
# Get response
res = musenet_generate(
    encode_musenet(midi_notes),
    length=60
)
completions = res.json()["completions"]

In [7]:
from musenet_api import play_musenet_response
from midi_api import setup_players

play_musenet_response(
    setup_players(),
    completions, 
    midi_notes,
    play_input=True
)

## Loop

In [8]:
from main import play_loop
play_loop(
    input_time=7500,
    generation_length=70
)

Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
GETTING INPUT
using input_id :1:
145420
Time is up
Getting reponse...
Took 8.53s
Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
GETTING INPUT
using input_id :1:
173048
Time is up
Getting reponse...
Took 8.09s
Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
GETTING INPUT
using input_id :1:
197877


KeyboardInterrupt: 